# 1、定义工具

In [2]:
import os
import dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 加载环境变量
dotenv.load_dotenv()

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 查询Tavily搜索API
search = TavilySearchResults(max_results=1)

# 执行查询
res = search.invoke("今天2025年11月23日上海天气怎么样？")
print(res)

[{'title': '上海市在十一月2025 的天氣：氣溫與氣候 - Climate-Data.org', 'url': 'https://zh.climate-data.org/%E4%BA%9E%E6%B4%B2/%E4%B8%AD%E5%9B%BD/%E4%B8%8A%E6%B5%B7%E5%B8%82-890/r/%E5%8D%81%E4%B8%80%E6%9C%88-11/', 'content': '##\n\n# 上海市 在 十一月 的天氣情況\n\n您打算在 十一月 2025 到 上海市 度假，希望天氣宜人嗎？在這裡您可以找到 上海市 在 十一月 的所有天氣資訊：\n\n >>上海市 在 十一月 的天氣與氣候概況\n >>上海市 每月天氣 // 天氣平均值\n >>上海市 在 十一月 的天氣平均狀況\n >>上海市 的當前溫度及天氣預報\n\n在 十一月，上海市 在 06.11 記錄了當天最高溫，達到 21.5°C，而最低溫則出現在 30.11，為 6.7°C。\n月初三分之一期間，平均最高溫為 20°C，最低溫為 12.1°C。\n第二個三分之一期間最高溫相似，約為 17.3°C，最低溫則接近 10.4°C。\n最後幾天的最高溫為 15.3°C，最低溫為 10.6°C。\n最熱的連續10天期間為 1-10，平均最高溫達 15.7°C，\n而最冷的連續10天則落在 21-10，最低溫約為 12.7°C。 [...] | 18. 十一月 | 12 °C | 15 °C | 9 °C | 3.5 mm | 0.0 |\n| 19. 十一月 | 12 °C | 16 °C | 9 °C | 1.7 mm | 0.0 |\n| 20. 十一月 | 12 °C | 17 °C | 9 °C | 3.0 mm | 0.0 |\n| 21. 十一月 | 12 °C | 16 °C | 8 °C | 3.1 mm | 0.0 |\n| 22. 十一月 | 12 °C | 16 °C | 8 °C | 1.8 mm | 0.0 |\n| 23. 十一月 | 12 °C | 16 °C | 8 °C | 1.3 mm | 0.0 |\n| 24. 十一月 | 12 °C | 16 °C | 7 °C | 2.1 mm | 0.0 |\n| 25. 十一月 | 11 °

# 2、定义Retriever

Retriever是langchain库中的一个模块, 用于检索工具

根据上述查询结果中的某个URL中, 获取一些数据创建一个检索器

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

# 1、创建一个大模型实例
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

embedding_model = OpenAIEmbeddings()

# 2、加载HTML内容为一个文档对象
loader = WebBaseLoader("https://zh.wikipedia.org/wiki/%E7%8C%AB")

docs = loader.load()

# 3、分割文档
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

documents = splitter.split_documents(docs)

# 4、向量化：得到向量数据库对象
vector = FAISS.from_documents(documents, embedding_model)

# 5、创建检索其
retriever = vector.as_retriever()

# 测试检索结果
print(retriever.invoke("猫的特征")[0])

page_content='感官[编辑]
貓的感官適於狩獵。在哺乳類動物中，貓的聽覺、視覺、嗅覺、味覺、觸覺極敏銳。

視覺[编辑]
貓的瞳孔能縮得如線般狹小
貓的瞳孔變化
貓在晝間視覺縱不及人類，夜視能力與追蹤視覺上之活動物件卻相當出色，夜視能力是人類的六倍，雖然綜合色彩計算整體視覺系數則僅及人類的十分之一。貓的眼睛具有微光觀察能力，即使只有微弱月光都可分辨物件，有關光線入貓眼後可放大40至50倍處理，令貓具有夜間活動的能力，即使在黑暗的地下室貓咪依然能活動自如。強光下，貓會將瞳孔縮得如線般狹小，以減少對視網膜的傷害，但視野會因而縮窄。由於貓眼具備高幀與高分辨率視覺，故此電視機極微細之動靜對貓而言亦成逐格跳躍之畫面。[51]
另外，不只是光線會影響貓的瞳孔，感情也會。一般而言貓放鬆的時候瞳孔會縮小，而緊張時會放大。
貓的視網膜背面有一層藍綠色如熒光一般的薄膜（Tapetum Lucidum），可增加在暗處的視力。閃光中，貓眼能呈現各式各樣顔色。如同多數食肉動物，貓眼長在臉上朝正前方，賦予其遼闊的視野，單一貓眼視野為100度，雙眼視野為285度，加上頸旋靈活，總計視野比人類雙眼僅100度靈活得多。不過，貓僅能聚焦其前30厘米至3米之物件，相對人類而言屬大近視。
貓對三原色的辨識力很差，相對人類而言，貓有黃藍色盲且紅綠色辨成灰黃色，與狗相似。關於貓的夜視能力，生物學家發現牛磺酸對貓的視力起了很大作用，貓本身不能合成牛磺酸，必須由外攝取。缺乏牛磺酸，會使視力及夜視能力變差，影響夜間活動。據生物學家稱，貓經常捕鼠，是因老鼠體內含牛磺酸。[52][53]
紫外線可以通過貓的晶状体。
當四周光線微弱，貓會用感覺毛來改善行動力與感知能力。感覺毛主要分布於鼻子兩側、下巴、雙眼上方、兩頰也有數根。感覺毛可感受非常微弱的空氣波動，視野不清時也能協助辨識阻礙位置。鬍鬚尖端與雙耳連成一線，恰是身體能通過障礙的最小範圍，故貓可在黑夜中快速判斷地形能否通過。
貓有第三眼瞼，當貓眼睑張開時，眨眼時第三眼瞼會從旁稍微遮蓋眼睛。若貓生病，或是睡眠，笑著，此眼皮會縮回一部分。若貓長時間嶄露第三眼瞼，表示它的健康有問題。' metadata={'source': 'https://zh.wikipedia.org/wiki/%E7%8C%AB', 'title': '猫 - 维基百科，自由的百科全书'

# 3、创建工具、工具集

In [7]:
from langchain.tools.retriever import create_retriever_tool

# 创建一个工具来检索文档
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="wiki_search",
    description="搜索维基百科"
)

创建工具集

In [8]:
tools = [search, retriever_tool]

# 4、语言模型调用工具

In [11]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages.human import HumanMessage

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 获取大模型
model = ChatOpenAI(
    model="gpt-4o-mini"
)

# 模型绑定工具
model_with_tools = model.bind_tools(tools)

# 根据输入自动调用工具
messages = [HumanMessage(content="今天2025年11月23日上海天气怎么样")]
response = model_with_tools.invoke(messages)
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '2025年11月23日上海天气预报'}, 'id': 'call_tAAcwhRkXQegp2mfumafhlnS', 'type': 'tool_call'}]


# 5、创建Agent程序（使用通用方式）

以FUNCTION_CALL方式进行调用

In [12]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")

print(prompt.messages)

C:\Users\xiaok\.conda\envs\langchain310\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [13]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 创建Agent对象
agent = create_tool_calling_agent(model, tools, prompt)

# 创建AgentExecutor对象
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

# 6、运行Agent

In [14]:
print(agent_executor.invoke({"input": "猫的特征"}))



> Entering new AgentExecutor chain...

Invoking: `wiki_search` with `{'query': '猫的特征'}`


感官[编辑]
貓的感官適於狩獵。在哺乳類動物中，貓的聽覺、視覺、嗅覺、味覺、觸覺極敏銳。

視覺[编辑]
貓的瞳孔能縮得如線般狹小
貓的瞳孔變化
貓在晝間視覺縱不及人類，夜視能力與追蹤視覺上之活動物件卻相當出色，夜視能力是人類的六倍，雖然綜合色彩計算整體視覺系數則僅及人類的十分之一。貓的眼睛具有微光觀察能力，即使只有微弱月光都可分辨物件，有關光線入貓眼後可放大40至50倍處理，令貓具有夜間活動的能力，即使在黑暗的地下室貓咪依然能活動自如。強光下，貓會將瞳孔縮得如線般狹小，以減少對視網膜的傷害，但視野會因而縮窄。由於貓眼具備高幀與高分辨率視覺，故此電視機極微細之動靜對貓而言亦成逐格跳躍之畫面。[51]
另外，不只是光線會影響貓的瞳孔，感情也會。一般而言貓放鬆的時候瞳孔會縮小，而緊張時會放大。
貓的視網膜背面有一層藍綠色如熒光一般的薄膜（Tapetum Lucidum），可增加在暗處的視力。閃光中，貓眼能呈現各式各樣顔色。如同多數食肉動物，貓眼長在臉上朝正前方，賦予其遼闊的視野，單一貓眼視野為100度，雙眼視野為285度，加上頸旋靈活，總計視野比人類雙眼僅100度靈活得多。不過，貓僅能聚焦其前30厘米至3米之物件，相對人類而言屬大近視。
貓對三原色的辨識力很差，相對人類而言，貓有黃藍色盲且紅綠色辨成灰黃色，與狗相似。關於貓的夜視能力，生物學家發現牛磺酸對貓的視力起了很大作用，貓本身不能合成牛磺酸，必須由外攝取。缺乏牛磺酸，會使視力及夜視能力變差，影響夜間活動。據生物學家稱，貓經常捕鼠，是因老鼠體內含牛磺酸。[52][53]
紫外線可以通過貓的晶状体。
當四周光線微弱，貓會用感覺毛來改善行動力與感知能力。感覺毛主要分布於鼻子兩側、下巴、雙眼上方、兩頰也有數根。感覺毛可感受非常微弱的空氣波動，視野不清時也能協助辨識阻礙位置。鬍鬚尖端與雙耳連成一線，恰是身體能通過障礙的最小範圍，故貓可在黑夜中快速判斷地形能否通過。
貓有第三眼瞼，當貓眼睑張開時，眨眼時第三眼瞼會從旁稍微遮蓋眼睛。若貓生病，或是睡眠，笑著，此眼皮會縮回一部分。若貓長時間嶄露第三眼瞼，表示它的健康有問題。

^ 生物學特性和解剖生理

# 7、添加记忆

In [15]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}


# 调用指定session_id对应的history
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]


agent_with_chat_history = RunnableWithMessageHistory(
    runnable=agent_executor,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

response = agent_with_chat_history.invoke(
    {"input": "Hi，我的名字是Cyber"},
    config={"configurable": {"session_id": "123"}},
)

print(response)



> Entering new AgentExecutor chain...
你好，Cyber！很高兴认识你！有什么我可以帮助你的吗？

> Finished chain.
{'input': 'Hi，我的名字是Cyber', 'chat_history': [], 'output': '你好，Cyber！很高兴认识你！有什么我可以帮助你的吗？'}


In [16]:
response = agent_with_chat_history.invoke(
{"input": "我叫什么名字?"},
config={"configurable": {"session_id": "123"}},
)
print(response)



> Entering new AgentExecutor chain...
你的名字是Cyber。你还有什么想要了解或者讨论的内容吗？

> Finished chain.
{'input': '我叫什么名字?', 'chat_history': [HumanMessage(content='Hi，我的名字是Cyber', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，Cyber！很高兴认识你！有什么我可以帮助你的吗？', additional_kwargs={}, response_metadata={})], 'output': '你的名字是Cyber。你还有什么想要了解或者讨论的内容吗？'}


In [17]:
response = agent_with_chat_history.invoke(
{"input": "我叫什么名字?"},
config={"configurable": {"session_id": "4566"}},
)
print(response)



> Entering new AgentExecutor chain...
抱歉，我无法直接知道您的名字。如果您愿意，可以告诉我您的名字，或者您需要我帮助您做些什么！

> Finished chain.
{'input': '我叫什么名字?', 'chat_history': [], 'output': '抱歉，我无法直接知道您的名字。如果您愿意，可以告诉我您的名字，或者您需要我帮助您做些什么！'}
